# NLP tasks with a simple interface 🗞️
Load your HF API key and relevant Python libraries.

In [9]:
%pip install huggingface_hub
 


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
%pip install requests


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
import os
import io
from IPython.display import Image, display, HTML
# from PIL import Image
import base64 
from dotenv import load_dotenv, find_dotenv
load_dotenv() # read local .env file
hf_api_key = os.getenv("HF_API_KEY")

In [19]:
# Helper function
import requests, json

#Summarization endpoint
def get_completion(inputs, parameters=None,ENDPOINT_URL=os.environ['HF_API_SUMMARY_BASE']): 
    headers = {
      "Authorization": f"Bearer {hf_api_key}",
      "Content-Type": "application/json"
    }
    data = { "inputs": inputs }
    if parameters is not None:
        data.update({"parameters": parameters})
    response = requests.request("POST",
                                ENDPOINT_URL, headers=headers,
                                data=json.dumps(data)
                               )
    return json.loads(response.content.decode("utf-8"))
  

## Building a text summarization app

In [13]:
import requests
import json
import os

def get_completion(inputs):
    ENDPOINT_URL = os.getenv('HF_API_SUMMARY_BASE')
    API_KEY = os.getenv('HF_API_KEY')

    if not API_KEY:
        raise ValueError("Missing Hugging Face API Key!")

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    data = {"inputs": inputs}

    response = requests.post(ENDPOINT_URL, headers=headers, json=data)

    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}, {response.text}"

text = """The tower is 324 metres (1,063 ft) tall, about the same height
as an 81-storey building, and the tallest structure in Paris. 
Its base is square, measuring 125 metres (410 ft) on each side. 
During its construction, the Eiffel Tower surpassed the Washington 
Monument to become the tallest man-made structure in the world,
a title it held for 41 years until the Chrysler Building
in New York City was finished in 1930. It was the first structure 
to reach a height of 300 metres."""

print(get_completion(text))


[{'summary_text': 'The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building. Its base is square, measuring 125 metres (410 ft) on each side. It was the first structure in the world to reach a height of 300 metres.'}]


In [20]:
text = ('''The tower is 324 metres (1,063 ft) tall, about the same height
        as an 81-storey building, and the tallest structure in Paris. 
        Its base is square, measuring 125 metres (410 ft) on each side. 
        During its construction, the Eiffel Tower surpassed the Washington 
        Monument to become the tallest man-made structure in the world,
        a title it held for 41 years until the Chrysler Building
        in New York City was finished in 1930. It was the first structure 
        to reach a height of 300 metres. Due to the addition of a broadcasting 
        aerial at the top of the tower in 1957, it is now taller than the 
        Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the 
        Eiffel Tower is the second tallest free-standing structure in France 
        after the Millau Viaduct.''')

get_completion(text)

[{'summary_text': 'The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building. Its base is square, measuring 125 metres (410 ft) on each side. It is the second tallest free-standing structure in France after the Millau Viaduct.'}]

### Getting started with Gradio `gr.Interface` 

In [18]:
import gradio as gr
def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']
    
gr.close_all()
demo = gr.Interface(fn=summarize, inputs="text", outputs="text")
demo.launch(share=True, server_port=7860)

Closing server running on port: 7860
Closing server running on port: 7861
Closing server running on port: 7861
Closing server running on port: 7860
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://f718b3fce46bfb3484.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [16]:
import gradio as gr

def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']

gr.close_all()
demo = gr.Interface(fn=summarize, 
                    inputs=[gr.Textbox(label="Text to summarize", lines=6)],
                    outputs=[gr.Textbox(label="Result", lines=3)],
                    title="Text summarization with bart-large-cnn",
                    description="Summarize any text using the `facebook/bart-large-cnn` model under the hood!"
                   )
demo.launch(share=True, server_port=7861)

Closing server running on port: 7860
Closing server running on port: 7861
Closing server running on port: 7860
* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://edbaa9f8d15bf85609.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Building a Named Entity Recognition app

We are using this [Inference Endpoint](https://huggingface.co/inference-endpoints) for `dslim/bert-base-NER`, a 108M parameter fine-tuned BART model on the NER task.

### How about running it locally?

```py
from transformers import pipeline

get_completion = pipeline("ner", model="dslim/bert-base-NER")

def ner(input):
    output = get_completion(input)
    return {"text": input, "entities": output}
    
```

In [21]:
API_URL = os.environ['HF_API_NER_BASE'] #NER endpoint
text = "My name is Andrew, I'm building DeepLearningAI and I live in California"
get_completion(text, parameters=None, ENDPOINT_URL= API_URL)

[{'entity_group': 'PER',
  'score': 0.9990624785423279,
  'word': 'Andrew',
  'start': 11,
  'end': 17},
 {'entity_group': 'ORG',
  'score': 0.8960502743721008,
  'word': 'DeepLearningAI',
  'start': 32,
  'end': 46},
 {'entity_group': 'LOC',
  'score': 0.999692440032959,
  'word': 'California',
  'start': 61,
  'end': 71}]

#### gr.interface()
- Notice below that we pass in a list `[]` to `inputs` and to `outputs` because the function `fn` (in this case, `ner()`, can take in more than one input and return more than one output.
- The number of objects passed to `inputs` list should match the number of parameters that the `fn` function takes in, and the number of objects passed to the `outputs` list should match the number of objects returned by the `fn` function.

In [22]:
def ner(input):
    output = get_completion(input, parameters=None, ENDPOINT_URL=API_URL)
    return {"text": input, "entities": output}

gr.close_all()
demo = gr.Interface(fn=ner,
                    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
                    outputs=[gr.HighlightedText(label="Text with entities")],
                    title="NER with dslim/bert-base-NER",
                    description="Find entities using the `dslim/bert-base-NER` model under the hood!",
                    allow_flagging="never",
                    #Here we introduce a new tag, examples, easy to use examples for your application
                    examples=["My name is Andrew and I live in California", "My name is Poli and work at HuggingFace"])
demo.launch(share=True, server_port=int(os.environ['PORT3']))

Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7861
Closing server running on port: 7860
Closing server running on port: 7861


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gradio/interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://69b9c3eff0b30ea4ea.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Adding a helper function to merge tokens

In [25]:
def merge_tokens(tokens):
    merged_tokens = []
    for token in tokens:
        if merged_tokens and token['entity_group'].startswith('I-') and merged_tokens[-1]['entity_group'].endswith(token['entity_group'][2:]):
            # If current token continues the entity of the last one, merge them
            last_token = merged_tokens[-1]
            last_token['word'] += token['word'].replace('##', '')
            last_token['end'] = token['end']
            last_token['score'] = (last_token['score'] + token['score']) / 2
        else:
            # Otherwise, add the token to the list
            merged_tokens.append(token)

    return merged_tokens

def ner(input):
    output = get_completion(input, parameters=None, ENDPOINT_URL=API_URL)
    merged_tokens = merge_tokens(output)
    return {"text": input, "entities": merged_tokens}

gr.close_all()
demo = gr.Interface(fn=ner,
                    inputs=[gr.Textbox(label="Text to find entities", lines=5)],
                    outputs=[gr.HighlightedText(label="Text with entities")],
                    title="NER with dslim/bert-base-NER",
                    description="Find entities using the `dslim/bert-base-NER` model under the hood!",
                    allow_flagging="never",
                    examples=["My name is Andrew, I'm building DeeplearningAI and I live in California", "My name is Poli, I live in Vienna and work at HuggingFace"])

demo.launch(share=True, server_port=int(os.environ['PORT4']))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gradio/interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7861
Closing server running on port: 7860
Closing server running on port: 7863
Closing server running on port: 7861
Closing server running on port: 7862
* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://32c5b06902f5bd992d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [26]:
gr.close_all()

Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7863
Closing server running on port: 7861
Closing server running on port: 7860
Closing server running on port: 7863
Closing server running on port: 7861
Closing server running on port: 7862
